# Milestone 3 - T5_base

In [1]:
dataset = ["""{
    "content": "世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、#idiom#。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、#idiom#、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。",
    "realCount": 2,
    "groundTruth": ["通力合作", "有的放矢"],
    "candidates": [
        ["凭空捏造", "高头大马", "通力合作", "同舟共济", "和衷共济", "蓬头垢面", "紧锣密鼓"],
        ["叫苦连天", "量体裁衣", "金榜题名", "百战不殆", "知彼知己", "有的放矢", "风流才子"]
    ]
}"""]
s = "世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、通力合作。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、</s>、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。可选成语有：叫苦连天 | 量体裁衣 | 金榜题名 | 百战不殆 | 知彼知己 | 风流才子，请选出最佳的一项。"

In [2]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 273.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 KB 301.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 313.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 274.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 223.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.2/701.2 KB 295.9 kB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 400.1 kB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
import json
import pandas as pd
import os
import torch
from torch import cuda, nn, optim
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np

# from google.colab import drive
# drive.mount('/content/drive/')
path = './'

# Load model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("mxmax/Chinese_Chat_T5_Base")
model = AutoModelForSeq2SeqLM.from_pretrained("mxmax/Chinese_Chat_T5_Base") 

device = 'cuda' if cuda.is_available() else 'cpu'
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

# Fine-tuning model 

## Data preprocessing

In [ ]:
def prompt1(text, candidates, choice):
    input_text = "请从（）里选择出最合适的成语: " + text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('#idiom#', "（" + candidates_str + "）", 1) 
    return input_text

In [5]:
def prompt2(text, candidates, choice):
    input_text = text.replace('#idiom#', "_")
    input_text = "请依次从（）选择出最合适的成语填入_: " + input_text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('（）', "（" + candidates_str + "）（）", 1)
    input_text = input_text.replace("（）", "")
    return input_text

In [ ]:
def prompt3(text, candidates, choice):
    ex = '''选择：[[“凭空捏造“, “高头大马“, “通力合作“, “同舟共济“, “和衷共济“, “蓬头垢面“, “紧锣密鼓“],[“叫苦连天“, “量体裁衣“, “金榜题名“, “百战不殆“, “知彼知己“, “有的放矢“, “风流才子“]]\n输入：“世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得“，就需要各方面密切配合、#idiom#。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、#idiom#、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。”\n输出：通力合作, 有的放矢\n'''
    input_text = ex
    input_text = input_text + f"选择：{candidates}\n输入：{text}\n输出："
    return input_text

In [6]:
def preprocess_data(data, prompt):
    input_texts = []
    labels = []

    for example in data:
        example = json.loads(example)
        
        input_text = example['content']
        ground_truth = example['groundTruth']
        candidates = example['candidates']
        
        input_text = prompt(input_text, candidates, ground_truth)
        input_texts.append(input_text)
        labels.append('、'.join(ground_truth))

    inputs = tokenizer(text=input_texts, return_token_type_ids=False)
    labels = tokenizer(labels, return_token_type_ids=False)
    return inputs, labels

In [7]:
# def preprocess_data(data):
#     input_texts = []
#     labels = []

#     for example in data:
#         example = json.loads(example)
        
#         input_text = example['content']
#         ground_truth = example['groundTruth']
#         candidates = example['candidates']

#         for i, idiom in enumerate(ground_truth):
#             input_text = "请从（）里选择出最合适的成语: " + input_text
#             candidates_str = '|'.join([c for c in candidates[i]])
#             input_text = input_text.replace('#idiom#', "（" + candidates_str + "）", 1)

#         input_texts.append(input_text)
#         labels.append('、'.join(ground_truth))

#     inputs = tokenizer(text=input_texts, return_token_type_ids=False)
#     labels = tokenizer(labels, return_token_type_ids=False)
#     return inputs, labels


In [8]:
# Load the Chinese Idioms dataset
train_data_file = path+'data/train_15000.txt'
val_data_file = path+'data/dev_2000.txt'



with open(train_data_file, encoding='utf-8', errors='ignore') as f:
    train_data = f.readlines()

with open(val_data_file, encoding='utf-8', errors='ignore') as f:
    val_data = f.readlines()

train_inputs, train_labels = preprocess_data(train_data, prompt2)
val_inputs, val_labels = preprocess_data(val_data, prompt2)

# train_inputs = preprocess_data(train_data)
# val_inputs = preprocess_data(val_data)


In [9]:
print(len(train_inputs))
print(train_inputs[0])
print(train_inputs[0].ids)
print(train_inputs[0].type_ids)
print(train_inputs[0].tokens)
print(train_inputs[0].offsets)
print(train_inputs[0].attention_mask)
print(train_inputs[0].special_tokens_mask)
print(train_inputs[0].overflowing)

# print("---------------------------------")



2
Encoding(num_tokens=151, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[12, 480, 9434, 57, 21, 25846, 291, 891, 4800, 1013, 2778, 146, 5684, 4800, 503, 237, 30846, 1954, 4800, 1676, 1313, 2903, 2330, 4800, 4453, 10765, 4800, 2050, 151, 36, 1744, 4800, 241, 38, 1546, 2862, 19, 367, 100, 134, 6064, 10581, 4034, 291, 3336, 20, 12, 3336, 10, 2936, 3441, 346, 991, 4115, 11793, 9, 5488, 24574, 94, 6, 1580, 1195, 31561, 5349, 6523, 145, 203, 364, 10, 222, 76, 1368, 9, 169, 123, 3132, 9, 70, 1335, 5488, 12963, 2847, 6, 107, 175, 683, 512, 3762, 38, 13028, 10, 5517, 9, 70, 8164, 7043, 10, 241, 362, 21258, 501, 4659, 43, 13758, 15779, 9, 295, 634, 43, 18, 23562, 1102, 2257, 9, 5406, 3733, 35, 7344, 27860, 5151, 6, 12549, 2810, 1193, 6014, 1003, 80, 22, 19188, 21236, 9, 14, 1010, 3145, 932, 596, 1359, 10328, 113, 2337, 2625, 27, 29527, 5635, 716, 1639, 5635, 1204, 69, 6, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
class IdiomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        input_ids = self.inputs['input_ids'][idx]
        attention_mask = self.inputs['attention_mask'][idx]
        # target_ids = self.inputs['input_ids'][idx]

        target_ids = self.labels['input_ids'][idx]
        target_attention_mask = self.labels['attention_mask'][idx]
        return {"input_ids": input_ids, "attention_mask":attention_mask, "label_ids":target_ids}


def collate_fn(batch):
    batch_input = [torch.LongTensor(example['input_ids']) for example in batch]
    batch_label = [torch.LongTensor(example['label_ids']) for example in batch]
    batch_mask = [torch.LongTensor(example['attention_mask']) for example in batch]

    padded_batch_input_ids = pad_sequence(batch_input, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_batch_label = pad_sequence(batch_label, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_batch_att_mask = pad_sequence(batch_mask, batch_first=True, padding_value=0)

    return {"input_ids": padded_batch_input_ids, "attention_mask": padded_batch_att_mask, "labels": padded_batch_label}

def to_device(data, device):
    new_data = {}
    for k in data:
        new_data[k] = data[k].to(device)
    return new_data

In [11]:
train_dataset = IdiomDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)

val_dataset = IdiomDataset(val_inputs, val_labels)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn, shuffle=False)


## Train model

In [12]:
def train(model:nn.Module, train_loader:DataLoader, optimizer:optim.Optimizer, log_step=100):
    model.train()
    epoch_loss = 0.0
    log_loss = 0.0
    for idx, batch in enumerate(train_loader):
        # print("idx:", idx, ", log_step: ", log_step)
        optimizer.zero_grad()
        batch = to_device(batch, device)
        loss = model(**batch).loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        log_loss += loss.item()
        if idx % log_step == 0:
            print(f"Train Step: {idx} Loss: {log_loss / log_step}")
            log_loss = 0.0
    return epoch_loss / len(train_loader)
        

@torch.no_grad()
def evaluate(model:nn.Module, eval_loader:DataLoader):
    eval_loss = 0.0
    correct = 0
    total = 0
    model.eval()
    print("eval_loader len:", len(eval_loader))
    for batch in eval_loader:
        batch = to_device(batch, device)
        output = model(**batch)
        loss = output.loss
        eval_loss += loss.item()
        pred = output.logits.argmax(-1)
        label = batch["labels"]
        correct += torch.where(label!=0, pred==label, 0).sum().item()
        total += torch.sum(label!=0).item()

    eval_acc = correct / total
    eval_loss = eval_loss / len(eval_loader) 
    print(total, correct)
    return eval_acc, eval_loss

In [ ]:
epoches = 5
optimizer = optim.Adam(model.parameters(), lr=2e-5)
model.train()

for epoch in range(1, epoches+1):
    print(f"Training Epoch {epoch}")
    train_loss = train(model, train_loader, optimizer)
    print(f"Epoch {epoch} Training Loss: {train_loss}")
    eval_acc, eval_loss = evaluate(model, val_loader)
    print(f"Epoch {epoch} Eval Acc: {eval_acc}; Eval Loss: {eval_loss}")
torch.save(model.state_dict(), path+"T5_base_prompt2_ckpt.pt")

Training Epoch 1
Train Step: 0 Loss: 0.0774147367477417
Train Step: 100 Loss: 4.605428206920624
Train Step: 200 Loss: 1.214688639640808
Train Step: 300 Loss: 0.7078378131985664
Train Step: 400 Loss: 0.6148210477828979
Train Step: 500 Loss: 0.5513512593507767
Train Step: 600 Loss: 0.5156533122062683
Train Step: 700 Loss: 0.4913174277544022
Train Step: 800 Loss: 0.45750915095210076
Train Step: 900 Loss: 0.45476491943001746
Train Step: 1000 Loss: 0.4370992037653923
Train Step: 1100 Loss: 0.42925464391708373
Train Step: 1200 Loss: 0.4599393877387047
Train Step: 1300 Loss: 0.4186902716755867
Train Step: 1400 Loss: 0.4271076732873917
Train Step: 1500 Loss: 0.4149249581992626
Train Step: 1600 Loss: 0.41327753946185114
Train Step: 1700 Loss: 0.39146889820694925
Train Step: 1800 Loss: 0.408329097032547
Epoch 1 Training Loss: 0.7354668735504151
eval_loader len: 250
12693 11003
Epoch 1 Eval Acc: 0.8668557472622705; Eval Loss: 0.283528157889843
Training Epoch 2
Train Step: 0 Loss: 0.00264148205518

## Evaluation

In [17]:
@torch.no_grad()
def fill_idiom(model, loader):

    all_preds = []
    all_labels = []
    model.eval()
    for batch in loader:
        batch = to_device(batch, device)
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, return_dict_in_generate=True, pad_token_id=0, max_length=512, top_k=15)
        truncated_outputs = []

        decode_texts = tokenizer.batch_decode([l[l != 0] for l in outputs["sequences"]])
        gold_texts = tokenizer.batch_decode([l[l != 0] for l in labels])

        for gold, decode in zip(gold_texts, decode_texts):
            l = set(gold.replace(" ", "").replace("</s>", "").split("、"))
            p = set(decode.replace(" ", "").replace("</s>", "").split("、"))
            all_labels.append(l)
            all_preds.append(p)
        # print(decode_texts)
        # print(gold_texts)
        # break
    
    return all_preds, all_labels

def f1_score(sys, gold):
    tp = 0
    t = 0
    p = 0
    for s, g in zip(sys, gold):
        t += len(g)
        p += len(s)
        tp += len(g & s)
    precision = tp / p if p != 0 else 0
    recall = tp / t if t != 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return precision, recall, f1, tp

In [26]:
# validation set
model.load_state_dict(torch.load(path+"T5_base_prompt2_ckpt.pt", map_location=device))
sys, gold = fill_idiom(model, val_loader)
p, r, f1, tp = f1_score(sys, gold)

print(f"Accuracy for Validation set is {f1}")
print(f"Accuracy for Validation set is {tp}")

Accuracy for Validation set is 0.5602851750891172
Accuracy for Validation set is 1336


In [23]:
# Load the Chinese Idioms dataset For Test set
test_data_file = path+'data/test_2000.txt'

with open(test_data_file, encoding='utf-8', errors='ignore') as f:
    test_data = f.readlines()

test_inputs, test_labels  = preprocess_data(test_data,prompt2)
test_dataset = IdiomDataset(test_inputs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [24]:
# test set
model.load_state_dict(torch.load(path+"T5_base_prompt2_ckpt.pt", map_location=device))
sys, gold = fill_idiom(model, test_loader)
p, r, f1, tp = f1_score(sys, gold)

print((f"F1 score for Test set is {f1}"))
print(f"Accuracy for Test set is {tp}")

F1 score for Test set is 0.4146023468057366
Accuracy for Test set is 954


In [25]:
print(tokenizer.decode([12]))
tokenizer.decode([12, 31200, 2057, 1498, 1278, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

'嗤恶痛绝</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Prompt engineering

## Load model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("mxmax/Chinese_Chat_T5_Base")
model = AutoModelForSeq2SeqLM.from_pretrained("mxmax/Chinese_Chat_T5_Base") 

device = 'cuda' if cuda.is_available() else 'cpu'
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

## Data processing

In [123]:
def prompt1(text, candidates, choice):
    input_text = "请从（）里选择出最合适的成语: " + text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('#idiom#', "（" + candidates_str + "）", 1) 
    return input_text

In [6]:
def prompt2(text, candidates, choice):
    input_text = "请依次从（）选择出最合适的成语填入#idiom#: " + text
    for i in range(len(choice)):
        candidates_str = '|'.join([c for c in candidates[i]])
        input_text = input_text.replace('（）', "（" + candidates_str + "）（）", 1)
    input_text = input_text.replace("（）", "")
    return input_text

In [125]:
def prompt3(text, candidates, choice):
    ex = '''选择：[[“凭空捏造“, “高头大马“, “通力合作“, “同舟共济“, “和衷共济“, “蓬头垢面“, “紧锣密鼓“],[“叫苦连天“, “量体裁衣“, “金榜题名“, “百战不殆“, “知彼知己“, “有的放矢“, “风流才子“]]\n输入：“世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得“，就需要各方面密切配合、#idiom#。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、#idiom#、灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。”\n输出：通力合作, 有的放矢\n'''
    input_text = ex
    input_text = input_text + f"选择：{candidates}\n输入：{text}\n输出："
    return input_text

In [7]:
def preprocess_data(data, prompt):
    input_texts = []
    labels = []

    for example in data:
        example = json.loads(example)
        
        input_text = example['content']
        ground_truth = example['groundTruth']
        candidates = example['candidates']
        
        input_text = prompt(input_text, candidates, ground_truth)
        input_texts.append(input_text)
        labels.append('、'.join(ground_truth))

    inputs = tokenizer(text=input_texts, return_token_type_ids=False)
    labels = tokenizer(labels, return_token_type_ids=False)
    return inputs, labels

In [127]:
# Load the Chinese Idioms dataset For Test set
test_data_file = path+'data/test_2000.txt'
with open(test_data_file, encoding='utf-8', errors='ignore') as f:
    test_data = f.readlines()

In [134]:
test_inputs1, test_labels1 = preprocess_data(test_data, prompt1)
test_dataset1 = IdiomDataset(test_inputs1, test_labels1)
test_loader1 = DataLoader(test_dataset1, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [135]:
test_inputs2, test_labels2 = preprocess_data(test_data, prompt2)
test_dataset2 = IdiomDataset(test_inputs2, test_labels2)
test_loader2 = DataLoader(test_dataset2, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [136]:
test_inputs3, test_labels3  = preprocess_data(test_data, prompt3)
test_dataset3 = IdiomDataset(test_inputs3, test_labels3)
test_loader3 = DataLoader(test_dataset3, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [131]:
print(test_inputs2[0].tokens)
print(test_inputs3[0].tokens)

['▁', '请', '依次', '从', '(', '旷', '日', '持久', '|', '公正', '廉洁', '|', '苦', '口', '婆', '心', '|', '现身', '说法', '|', '白', '日', '做梦', '|', '深入浅出', '|', '肺', '腑', '之', '言', ')', '选择', '出', '最', '合适的', '成语', '填', '入', '#', 'id', 'io', 'm', '#', ':', '▁', '只要', '路过', '的', '旅客', '稍有', '迟', '疑', ',', '或者', '对他们', '的宣传', '单', '多', '看', '几', '眼', ',', '基本上', '这个', '旅客', '就别', '想', '轻松', '脱', '身', '了', ',', '记者', '就在', '9', '月', '3', '日', '接', '站', '时', '目睹', '了', '这样', '一幕', ':', '一个', '学生', '接', '过', '招生', '人员', '递', '来', '的宣传', '单', ',', '只是', '问', '了一下', '“', '你们', '学校', '有没有', '分数', '要求', '?”', '两个', '招生', '人员', '就', '“', '白话', '”', '开了', ',', '一个', '表示', '分数', '都', '好', '说', ',', '只要有', '好', '学', '的精神', ';', '另一个', '则', '#', 'id', 'io', 'm', '#', ',', '大', '讲', '自己', '选择', '的专业', '现在', '收获', '颇', '丰', ';', '最后', '在', '招生', '人员', '“', '我们学校', '毕业后', '可以', '完全', '解决', '就业', '”', '的', '忽悠', '下', ',', '这个', '学生', '旅客', '被', '他们', '拉', '上了', '到', '校', '参观', '的', '班车', '。', '</s>']
['▁', '选择', ':', '[', 

In [137]:
# prompt engineering for test set (prompt1)
model.load_state_dict(torch.load(path+"T5_base_prompt2_ckpt.pt.pt", map_location=device))
sys1, gold1 = fill_idiom(model, test_loader1)
p1, r1, f11, tp1 = f1_score(sys1, gold1)
print((f"F1 score for Test set is {f11}"))
print(f"Accuracy for Test set is {tp1}")

# prompt engineering for test set (prompt2)
sys2, gold2 = fill_idiom(model, test_loader2)
p2, r2, f12, tp2 = f1_score(sys2, gold2)
print((f"F1 score for Test set is {f12}"))
print(f"Accuracy for Test set is {tp2}")

# prompt engineering for test set (prompt3)
sys3, gold3 = fill_idiom(model, test_loader3)
p3, r3, f13, tp3 = f1_score(sys3, gold3)
print((f"F1 score for Test set is {f13}"))
print(f"Accuracy for Test set is {tp3}")

F1 score for Test set is 0.5310709563684443
Accuracy for Test set is 1205
F1 score for Test set is 0.39724553475360447
Accuracy for Test set is 923
F1 score for Test set is 0.004140786749482402
Accuracy for Test set is 10


In [4]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
# import torch
# from torch import cuda
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained("mxmax/Chinese_Chat_T5_Base")
# model = AutoModelForSeq2SeqLM.from_pretrained("mxmax/Chinese_Chat_T5_Base") 
# device = 'cuda' if cuda.is_available() else 'cpu'
# model.to(device)
def postprocess(text):
    return text.replace(".", "").replace('</>','')

def answer_fn(text, top_k=50):
    encoding = tokenizer(text=[text], truncation=True, padding=True, max_length=256, return_tensors="pt").to(device) 
    out = model.generate(**encoding, return_dict_in_generate=True, output_scores=False, max_length=512,temperature=0.5,do_sample=True,repetition_penalty=3.0 ,top_k=top_k)
    result = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)
    return postprocess(result[0])

x1 = """世锦赛的整体水平远高于亚洲杯，要如同亚洲杯那样“鱼与熊掌兼得”，就需要各方面密切配合、（凭空捏造|高头大马|通力合作|同舟共济|和衷共济|蓬头垢面|紧锣密鼓）。作为主帅的俞觉敏，除了得打破保守思想，敢于破格用人，还得巧于用兵、(叫苦连天|量体裁衣|金榜题名|百战不殆|知彼知己|风流才子)、
灵活排阵，指挥得当，力争通过比赛推新人、出佳绩、出新的战斗力。"""

# y1 = ["高头大马", "叫苦连天"]

result=answer_fn(x1, top_k=50)
print("模型生成:",result)
print('*'*100)


模型生成: 在亚洲杯上，中国选手俞觉敏的表现非常出色。她不仅发挥了出色的表现力和团队合作能力外也取得了很多成功：通过比赛推新人、出佳绩以及新战斗力等各方面的配合来达到自己的目的并取得胜利;同时她也能够与其他国家的比赛中保持联系并与对手进行交流并获得经验教训等等都是她的成就之一。
****************************************************************************************************
